In [1]:
### EXAMPLE

# address = '145 JEFFERSON ST., 94133'
# url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

# response = requests.get(url).json()
# print(response[0]["lat"])
# print(response[0]["lon"])

In [4]:
import requests
import urllib.parse
import pandas as pd

focus_df = pd.read_csv('/Users/Gustav/Downloads/socialdataData.csv')
# drop na in postal code
focus_df = focus_df.dropna(subset=['Postal Code'])

In [5]:
focus_df

,Parcel(s),Building Name,Building Address,Postal Code,Floor Area,Category,Property Type - Self Selected,PIM Link,Year Built,Energy Audit Due Date,...,2015 Benchmark Status,2015 Reason for Exemption,2014 Benchmark Status,2014 Reason for Exemption,2013 Benchmark Status,2013 Reason for Exemption,2012 Benchmark Status,2012 Reason for Exemption,2011 Benchmark Status,2011 Reason for Exemption
0,0010/001,2801 Leavenworth Street,2801 LEAVENWORTH ST,94109.0,133675.0,Commercial,Office,https://sfplanninggis.org/PIM/?tab=Property&se...,1907.0,04/01/2024,...,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN
1,0010/002,Argonaut Hotel-SV,495 JEFFERSON ST,94109.0,180000.0,Commercial,Hotel,https://sfplanninggis.org/PIM/?tab=Property&se...,1907.0,04/01/2025,...,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN
2,0011/007,Anchorage Square (2800 Leavenworth),2800 LEAVENWORTH ST,94133.0,485846.0,Commercial,Strip Mall,https://sfplanninggis.org/PIM/?tab=Property&se...,0.0,04/01/2024,...,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN,Violation - Insufficient Data,NaN
3,0011/008,Anchorage Garage,500 BEACH ST,94133.0,198525.0,Commercial,Strip Mall,https://sfplanninggis.org/PIM/?tab=Property&se...,1974.0,04/01/2024,...,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN
4,0012/001,211 Jefferson St,2739 TAYLOR ST,94133.0,40052.0,Commercial,Retail Store,https://sfplanninggis.org/PIM/?tab=Property&se...,1953.0,04/01/2025,...,Complied,NaN,Exempt,Tenant Turnover,Exempt,Tenant Turnover,Complied,NaN,Violation - Insufficient Data,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553,9900/045-D,Pier 45 - Shed D,45 PIER - SHED D,94133.0,64628.0,Commercial - Port Facility,Other,https://sfplanninggis.org/PIM/?tab=Property&se...,1900.0,04/01/2099,...,Exempt,Added After Benchmark Year,Exempt,Added After Benchmark Year,Exempt,Added After Benchmark Year,Exempt,Added After Benchmark Year,NaN,NaN
2554,9900/048,Pier 48 - Shed A,48 PIER - SHED A,94158.0,83852.0,Commercial - Port Facility,Other,https://sfplanninggis.org/PIM/?tab=Property&se...,1900.0,04/01/2099,...,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN
2555,9900/050,"Pier 50 - Shed A, B, and D",50 PIER,94158.0,108431.0,Commercial - Port Facility,Non-Refrigerated Warehouse,https://sfplanninggis.org/PIM/?tab=Property&se...,1900.0,04/01/2099,...,Complied,NaN,Complied,NaN,Violation - Did Not Report,NaN,Violation - Did Not Report,NaN,Violation - Did Not Report,NaN
2556,9900/050H,Pier 50 - Administration Building,401 Terry A Francois Blvd,94107.0,11701.0,Commercial - Port Facility,Office,https://sfplanninggis.org/PIM/?tab=Property&se...,NaN,04/01/2099,...,Violation - Did Not Report,NaN,Exempt,Added After Benchmark Year,Exempt,Added After Benchmark Year,Exempt,Added After Benchmark Year,NaN,NaN


In [14]:
lat = []
lng = []
failed_requests = []
counter = 0
print(len(focus_df))
for index, row in focus_df.iterrows():
    print(counter)
    counter += 1

    address = row['Building Address']
    postal_code = row['Postal Code']
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address + ', ' + str(int(postal_code))) + '?format=json'
    response = requests.get(url).json()
    if response == []:
        failed_requests.append(url)
        print(url)
        lat.append(0)
        lng.append(0)
        continue
    lat.append(response[0]['lat'])
    lng.append(response[0]['lon'])
print(len(lat))
print(len(lng))
print(len(failed_requests))

2522
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

In [15]:
failed_requests

['https://nominatim.openstreetmap.org/search/5%20EMBARCADERO%20CENTER%2C%2094111?format=json',
 'https://nominatim.openstreetmap.org/search/601%20OFARRELL%20ST%2C%2094109?format=json',
 'https://nominatim.openstreetmap.org/search/641%20OFARRELL%20ST%2C%2094109?format=json',
 'https://nominatim.openstreetmap.org/search/631%20OFARRELL%20ST%2C%2094109?format=json',
 'https://nominatim.openstreetmap.org/search/20%20OFARRELL%20ST%2C%2094108?format=json',
 'https://nominatim.openstreetmap.org/search/180%20OFARRELL%2C%2094102?format=json',
 'https://nominatim.openstreetmap.org/search/260%20OFARRELL%20ST%2C%2094102?format=json',
 'https://nominatim.openstreetmap.org/search/450%20OFARRELL%20ST%2C%2094102?format=json',
 'https://nominatim.openstreetmap.org/search/474%20OFARRELL%20ST%2C%2094102?format=json',
 'https://nominatim.openstreetmap.org/search/750%20OFARRELL%20ST%2C%2094109?format=json',
 'https://nominatim.openstreetmap.org/search/415%20OFARRELL%20ST%2C%2094102?format=json',
 'https://n

In [16]:
focus_df['lat'] = lat
focus_df['lng'] = lng

In [17]:
focus_df

,Parcel(s),Building Name,Building Address,Postal Code,Floor Area,Category,Property Type - Self Selected,PIM Link,Year Built,Energy Audit Due Date,...,2014 Benchmark Status,2014 Reason for Exemption,2013 Benchmark Status,2013 Reason for Exemption,2012 Benchmark Status,2012 Reason for Exemption,2011 Benchmark Status,2011 Reason for Exemption,lat,lng
0,0010/001,2801 Leavenworth Street,2801 LEAVENWORTH ST,94109.0,133675.0,Commercial,Office,https://sfplanninggis.org/PIM/?tab=Property&se...,1907.0,04/01/2024,...,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN,37.806973,-122.419034
1,0010/002,Argonaut Hotel-SV,495 JEFFERSON ST,94109.0,180000.0,Commercial,Hotel,https://sfplanninggis.org/PIM/?tab=Property&se...,1907.0,04/01/2025,...,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN,37.807188350000004,-122.42022810445624
2,0011/007,Anchorage Square (2800 Leavenworth),2800 LEAVENWORTH ST,94133.0,485846.0,Commercial,Strip Mall,https://sfplanninggis.org/PIM/?tab=Property&se...,0.0,04/01/2024,...,Complied,NaN,Complied,NaN,Complied,NaN,Violation - Insufficient Data,NaN,37.806997,-122.418828
3,0011/008,Anchorage Garage,500 BEACH ST,94133.0,198525.0,Commercial,Strip Mall,https://sfplanninggis.org/PIM/?tab=Property&se...,1974.0,04/01/2024,...,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN,37.8071297,-122.417931
4,0012/001,211 Jefferson St,2739 TAYLOR ST,94133.0,40052.0,Commercial,Retail Store,https://sfplanninggis.org/PIM/?tab=Property&se...,1953.0,04/01/2025,...,Exempt,Tenant Turnover,Exempt,Tenant Turnover,Complied,NaN,Violation - Insufficient Data,NaN,37.807654102040814,-122.41579612244898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553,9900/045-D,Pier 45 - Shed D,45 PIER - SHED D,94133.0,64628.0,Commercial - Port Facility,Other,https://sfplanninggis.org/PIM/?tab=Property&se...,1900.0,04/01/2099,...,Exempt,Added After Benchmark Year,Exempt,Added After Benchmark Year,Exempt,Added After Benchmark Year,NaN,NaN,0,0
2554,9900/048,Pier 48 - Shed A,48 PIER - SHED A,94158.0,83852.0,Commercial - Port Facility,Other,https://sfplanninggis.org/PIM/?tab=Property&se...,1900.0,04/01/2099,...,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN,0,0
2555,9900/050,"Pier 50 - Shed A, B, and D",50 PIER,94158.0,108431.0,Commercial - Port Facility,Non-Refrigerated Warehouse,https://sfplanninggis.org/PIM/?tab=Property&se...,1900.0,04/01/2099,...,Complied,NaN,Violation - Did Not Report,NaN,Violation - Did Not Report,NaN,Violation - Did Not Report,NaN,44.6597594,10.9001248
2556,9900/050H,Pier 50 - Administration Building,401 Terry A Francois Blvd,94107.0,11701.0,Commercial - Port Facility,Office,https://sfplanninggis.org/PIM/?tab=Property&se...,NaN,04/01/2099,...,Exempt,Added After Benchmark Year,Exempt,Added After Benchmark Year,Exempt,Added After Benchmark Year,NaN,NaN,37.77647240581685,-122.38832727200112


In [18]:
focus_df.to_csv('/Users/Gustav/Downloads/socialdataData_wlatlng.csv')